# Installing, Importing, Connecting to Database

In [ ]:
pip install matplotlib pymysql pandas SQLAlchemy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 1.5 MB/s eta 0:00:00


In [ ]:
import pymysql
import pandas as pd
from pandas.tseries.offsets import DateOffset
from sqlalchemy import create_engine
import statistics
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator, FuncFormatter
import textwrap
import numpy as np

In [ ]:
db =  {
    "hostname":"sbaloandata.c1y4g4kqkbn1.us-east-1.rds.amazonaws.com",
    "port":3306,
    "username": "sbadev1",
    "password": "SbaDevATL1!",
    "database": "sbaloan"
    }

connection_string = f"mysql+pymysql://{db['username']}:{db['password']}@{db['hostname']}:{db['port']}/{db['database']}"
engine = create_engine(connection_string)

In [ ]:
def res_df_input(origination_date, origination_date_range, engine, MinGrossApproval = None, MaxGrossApproval = None, MinimumTermInMonths = None, NaicsCode = None, BorrState = None, BorrZip = None, BorrCity = None, BankName = None, ):
    origination_date = pd.to_datetime(origination_date)
    end_date = origination_date + DateOffset(months=origination_date_range) - DateOffset(days=1)

    start_date_str = origination_date.strftime('%Y-%m-%d')
    end_date_str = end_date.strftime('%Y-%m-%d')

    query = f"""
        SELECT *
        FROM sba_loan_7a
        WHERE ApprovalDate BETWEEN '{start_date_str}' AND '{end_date_str}'
    """

    conditions = []

    if MinGrossApproval is not None:
        conditions.append(f"GrossApproval >= {MinGrossApproval}")

    if MaxGrossApproval is not None:
        conditions.append(f"GrossApproval <= {MaxGrossApproval}")

    if MinimumTermInMonths is not None:
        conditions.append(f"TermInMonths >= '{MinimumTermInMonths}'")

    # to get the sector on the buisness
    # can change easily for the full NaicsCode
    if NaicsCode is not None:
        conditions.append(f"SUBSTRING(NaicsCode, 1, 2) = '{NaicsCode[:2]}'")

    if BorrState is not None:
        conditions.append(f"BorrState = '{BorrState}'")

    if BorrZip is not None:
        conditions.append(f"BorrZip = '{BorrZip}'")

    if BorrCity is not None:
        conditions.append(f"BorrCity = '{BorrCity}'")

    if BankName is not None:
        conditions.append(f"BankName = '{BankName}'")

    if conditions:
        query += " AND " + " AND ".join(conditions)

    query += " ORDER BY ApprovalDate"

    res_df = pd.read_sql_query(query, engine)

    return res_df

# Basic Testing, PD/LGD functions

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)

# change the parameters accordingly
origination_date = '1990-09-01'
origination_date_range = 3   # range for the origination date
MinGrossApproval = None
MaxGrossApproval = None
MinimumTermInMonths = None   # minimum term of the loan
NaicsCode = None             #sector of NaicsCode (need string, not number)
BorrState = None
BorrZip = None
BorrCity = None
BankName = None

res_df = res_df_input(origination_date,
                      origination_date_range,
                      engine,
                      MinGrossApproval = MinGrossApproval,
                      MaxGrossApproval = MaxGrossApproval,
                      MinimumTermInMonths = MinimumTermInMonths,
                      NaicsCode = NaicsCode,
                      BorrState = BorrState,
                      BorrZip = BorrZip,
                      BorrCity = BorrCity,
                      BankName = BankName)
res_df

,LoanId,Program,BorrName,BorrStreet,BorrCity,BorrState,BorrZip,BankName,BankFDICNumber,BankNCUANumber,BankStreet,BankCity,BankState,BankZip,GrossApproval,SBAGuaranteedApproval,ApprovalDate,ApprovalFiscalYear,FirstDisbursementDate,DeliveryMethod,Subprogram,InitialInterestRate,FixedOrVariableInterestInd,TermInMonths,NaicsCode,NaicsDescription,FranchiseCode,FranchiseName,ProjectCounty,ProjectState,SBADistrictOffice,CongressionalDistrict,BusinessType,BusinessAge,LoanStatus,PaidInFullDate,ChargeOffDate,GrossChargeOffAmount,RevolverStatus,JobsSupported,SoldSecMrktInd
0,03de3f67-176b-4945-a081-2a0c5744c38f,7A,GBR INC. & GERALD H & BONITA,1103 SPRUCE,YANKTON,SD,57078,CorTrust Bank National Association,6063.0,NaN,100 E Havens Ave,MITCHELL,SD,57301,140000.0,116200.0,1990-10-01,1991,1991-01-31,OTH 7A,Guaranty,0.0,,180,442210,Floor Covering Stores,None,,YANKTON,SD,SOUTH DAKOTA DISTRICT OFFICE,00,INDIVIDUAL,,PIF,2005-07-31,None,0.0,0,0,Y
1,147226ef-6e73-4da3-bae5-d95a16aa5ae5,7A,MICRO-PACE COMPUTERS INC.,"604 N. COUNTRY FAIR DRIVE, S",CHAMPAIGN,IL,61821,CIBM Bank,11729.0,NaN,2913 W Kirby Ave,CHAMPAIGN,IL,61821,330000.0,280500.0,1990-10-01,1991,1990-10-31,OTH 7A,Guaranty,0.0,,84,421990,Other Miscellaneous Durable Goods Wholesalers,None,,CHAMPAIGN,IL,SPRINGFIELD BRANCH OFFICE,13,CORPORATION,,PIF,2005-07-31,None,0.0,0,0,Y
2,29da16b6-d830-4e5e-805c-3174cad94743,7A,CANDLELIGHT INN,2831 SW FAIRLAWN ROAD,TOPEKA,KS,66614,CoreFirst Bank & Trust,18100.0,NaN,3035 SW Topeka Blvd,TOPEKA,KS,66611,600000.0,510000.0,1990-10-01,1991,1990-10-31,OTH 7A,Guaranty,0.0,,180,,,None,,SHAWNEE,KS,KANSAS CITY DISTRICT OFFICE,01,PARTNERSHIP,,PIF,2005-07-31,None,0.0,0,0,
3,51def2b3-1e75-4992-9c84-819f858acaf8,7A,M.C. INTERNATIONAL INC,1100-B TUCKAHOE DRIVE,NASHVILLE,TN,37207,First Horizon Bank,4977.0,NaN,165 Madison Ave,MEMPHIS,TN,38103,96355.0,86720.0,1990-10-01,1991,1991-01-31,OTH 7A,Guaranty,0.0,,6,422330,"Women's, Children's, and Infants' Clothing and...",None,,DAVIDSON,TN,TENNESSEE DISTRICT OFFICE,05,CORPORATION,,PIF,2005-07-31,None,0.0,0,0,
4,6c6b7760-19c8-44b8-9501-7d8aeaab38cf,7A,Riv Inc.,36 CABOT ROAD,MERRIMACK,NH,03054,MISSINGMAINBANKID,NaN,NaN,,,,,120000.0,108000.0,1990-10-01,1991,None,CLP,Guaranty,0.0,,240,421830,Industrial Machinery and Equipment Wholesalers,None,,HILLSBOROUGH,NH,NEW HAMPSHIRE DISTRICT OFFICE,01,CORPORATION,,CANCLD,None,None,0.0,0,0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2790,f584032a-3e3d-45f3-aafe-6212cb375cf0,7A,INTERNATIONAL FILTER MANUFACTU,"2112 SCHRAM AVE., P.O. BOX 5",HILLSBORO,IL,62049,The Litchfield National Bank,3808.0,NaN,316 N State St,LITCHFIELD,IL,62056,76000.0,68400.0,1990-11-30,1991,None,OTH 7A,Guaranty,0.0,,84,,,None,,MONTGOMERY,IL,SPRINGFIELD BRANCH OFFICE,13,CORPORATION,,CANCLD,None,None,0.0,0,0,
2791,f5982a2b-4a0b-4229-8491-a6911202949b,7A,BELLINI JUNEVILE DESIGNER FURN,1126 E. PRADA COURT,SAN JOSE,CA,95120,Bank of Hope,26610.0,NaN,3200 Wilshire Blvd 1st Fl,LOS ANGELES,CA,90010,105000.0,94500.0,1990-11-30,1991,None,OTH 7A,Guaranty,0.0,,120,,,09700,BELLINI JUVENILE FURNITURE (7-,SANTA CLARA,CA,SAN FRANCISCO DISTRICT OFFICE,18,INDIVIDUAL,,CANCLD,None,None,0.0,0,0,
2792,faf75a94-161b-4570-8d3e-b3ca600735ed,7A,CUISINARY INC DBA ADRIANA'S BA,335 WEST 38 STREET,NEW YORK,NY,10018,Apple Bank for Savings,16068.0,NaN,1395 Northern Blvd,MANHASSET,NY,11030,100000.0,90000.0,1990-11-30,1991,1991-04-30,OTH 7A,Guaranty,0.0,,63,,,None,,NEW YORK,NY,NEW YORK DISTRICT OFFICE,10,CORPORATION,,CHGOFF,None,1999-01-27,119474.0,0,0,
2793,fb49bc34-188b-42b6-ab5a-92200c1ffb2e,7A,WINANS CONSTRUCTION INC.,"5515 DOYLE STREET, #9",EMERYVILLE,CA,94608,BMO Bank National Association,16571.0,NaN,320 S Canal St,Chicago,IL,60606,335600.0,268480.0,1990-11-30,1991,1991-01-31,PLP,Guaranty,0.0,,288,,,None,,ALAMEDA,CA,SAN FRANCISCO DISTRICT OFFICE,13,CORPORATION,,PIF,2005-07-31,None,0.0,0,0,Y


In [ ]:
res_df['ChargeOffDate'] = pd.to_datetime(res_df['ChargeOffDate'])
res_df['ApprovalDate'] = pd.to_datetime(res_df['ApprovalDate'])

def max_default_month(res_df):
    CHGOFF_loans = res_df[res_df['LoanStatus'] == 'CHGOFF'].dropna(subset=['ChargeOffDate', 'ApprovalDate'])

    if len(CHGOFF_loans) == 0:
        return 1

    difference_in_quarters = (
        CHGOFF_loans['ChargeOffDate'].dt.to_period('M').astype(int) -
        CHGOFF_loans['ApprovalDate'].dt.to_period('M').astype(int)
    )

    max_difference = difference_in_quarters.max(skipna=True) + 1

    if pd.isna(max_difference):
        return 0
    # needs some fixing; sometimes, max_difference is sometimes off due to rounding
    return int(max_difference)

def max_default_quarter(res_df):
    CHGOFF_loans = res_df[res_df['LoanStatus'] == 'CHGOFF'].dropna(subset=['ChargeOffDate', 'ApprovalDate'])

    if len(CHGOFF_loans) == 0:
        return 1

    difference_in_quarters = (
        CHGOFF_loans['ChargeOffDate'].dt.to_period('Q').astype(int) -
        CHGOFF_loans['ApprovalDate'].dt.to_period('Q').astype(int)
    )

    max_difference = difference_in_quarters.max(skipna=True) + 1
    # needs some fixing; sometimes, max_difference is sometimes off due to rounding
    return int(max_difference)

In [ ]:
def PD_month(res_df, max_default_months):
    total_loans = res_df.shape[0]
    PD_month_array = []
    CHGOFF_loans = res_df[res_df['LoanStatus'] == 'CHGOFF'].dropna(subset=['ChargeOffDate', 'ApprovalDate'])

    for default_month in range(1,max_default_months):
        default_period_start = CHGOFF_loans['ApprovalDate'] + DateOffset(months=default_month-1)
        default_period_end   = CHGOFF_loans['ApprovalDate'] + DateOffset(months=default_month)

        loans_already_defaulted = (CHGOFF_loans['ChargeOffDate'] < default_period_start).sum()
        defaulted_loans = ((CHGOFF_loans['ChargeOffDate'] >= default_period_start) & (CHGOFF_loans['ChargeOffDate'] < default_period_end)).sum()

        surviving_loans = total_loans - loans_already_defaulted
        probability_of_default = defaulted_loans / surviving_loans if surviving_loans > 0 else 0

        PD_month_array.append([
            int(default_month),
            float(probability_of_default),
            int(defaulted_loans),
            int(surviving_loans)
        ])

    return PD_month_array

def PD_quarter(res_df, max_default_quarters):
    total_loans = res_df.shape[0]
    PD_quarter_array = []
    CHGOFF_loans = res_df[res_df['LoanStatus'] == 'CHGOFF'].dropna(subset=['ChargeOffDate', 'ApprovalDate'])

    for default_quarter in range(1,max_default_quarters):
        default_period_start = CHGOFF_loans['ApprovalDate'] + DateOffset(months=(default_quarter - 1) * 3)
        default_period_end = CHGOFF_loans['ApprovalDate'] + DateOffset(months=default_quarter * 3)

        loans_already_defaulted = (CHGOFF_loans['ChargeOffDate'] < default_period_start).sum()
        defaulted_loans = ((CHGOFF_loans['ChargeOffDate'] >= default_period_start) & (CHGOFF_loans['ChargeOffDate'] < default_period_end)).sum()

        surviving_loans = total_loans - loans_already_defaulted
        probability_of_default = defaulted_loans / surviving_loans if surviving_loans > 0 else 0

        PD_quarter_array.append([
            int(default_quarter),
            float(probability_of_default),
            int(defaulted_loans),
            int(surviving_loans)
        ])

    return PD_quarter_array

In [ ]:
caption = ''

variables = [
    ("Origination Date", origination_date),
    ("Origination Date Range", origination_date_range),
    ("Min Gross Approval", MinGrossApproval),
    ("Max Gross Approval", MaxGrossApproval),
    ("Min Term (Months)", MinimumTermInMonths),
    ("NAICS Code", NaicsCode),
    ("Borrower State", BorrState),
    ("Borrower Zip", BorrZip),
    ("Borrower City", BorrCity),
    ("Bank Name", BankName)
]

for desc, var in variables:
    if var is not None:
        caption += f'{desc}: {var} | '

# Obtaining .csv's for preprocessed data (PD)

In [ ]:
# Reset
origination_date = '1990-10-01'
origination_date_range = 408
MinGrossApproval = None
MaxGrossApproval = None
MinimumTermInMonths = None
NaicsCode = None
BorrState = None
BorrZip = None
BorrCity = None
BankName = None

res_df = res_df_input(origination_date,
                      origination_date_range,
                      engine,
                      MinGrossApproval = MinGrossApproval,
                      MaxGrossApproval = MaxGrossApproval,
                      MinimumTermInMonths = MinimumTermInMonths,
                      NaicsCode = NaicsCode,
                      BorrState = BorrState,
                      BorrZip = BorrZip,
                      BorrCity = BorrCity,
                      BankName = BankName)

res_df['ChargeOffDate'] = pd.to_datetime(res_df['ChargeOffDate'])
res_df['ApprovalDate'] = pd.to_datetime(res_df['ApprovalDate'])
res_df = res_df.dropna(subset=['ApprovalDate'])

In [ ]:
res_df

,LoanId,Program,BorrName,BorrStreet,BorrCity,BorrState,BorrZip,BankName,BankFDICNumber,BankNCUANumber,BankStreet,BankCity,BankState,BankZip,GrossApproval,SBAGuaranteedApproval,ApprovalDate,ApprovalFiscalYear,FirstDisbursementDate,DeliveryMethod,Subprogram,InitialInterestRate,FixedOrVariableInterestInd,TermInMonths,NaicsCode,NaicsDescription,FranchiseCode,FranchiseName,ProjectCounty,ProjectState,SBADistrictOffice,CongressionalDistrict,BusinessType,BusinessAge,LoanStatus,PaidInFullDate,ChargeOffDate,GrossChargeOffAmount,RevolverStatus,JobsSupported,SoldSecMrktInd
0,03de3f67-176b-4945-a081-2a0c5744c38f,7A,GBR INC. & GERALD H & BONITA,1103 SPRUCE,YANKTON,SD,57078,CorTrust Bank National Association,6063.0,NaN,100 E Havens Ave,MITCHELL,SD,57301,140000.0,116200.0,1990-10-01,1991,1991-01-31,OTH 7A,Guaranty,0.0,,180,442210,Floor Covering Stores,None,,YANKTON,SD,SOUTH DAKOTA DISTRICT OFFICE,00,INDIVIDUAL,,PIF,2005-07-31,NaT,0.0,0,0,Y
1,147226ef-6e73-4da3-bae5-d95a16aa5ae5,7A,MICRO-PACE COMPUTERS INC.,"604 N. COUNTRY FAIR DRIVE, S",CHAMPAIGN,IL,61821,CIBM Bank,11729.0,NaN,2913 W Kirby Ave,CHAMPAIGN,IL,61821,330000.0,280500.0,1990-10-01,1991,1990-10-31,OTH 7A,Guaranty,0.0,,84,421990,Other Miscellaneous Durable Goods Wholesalers,None,,CHAMPAIGN,IL,SPRINGFIELD BRANCH OFFICE,13,CORPORATION,,PIF,2005-07-31,NaT,0.0,0,0,Y
2,29da16b6-d830-4e5e-805c-3174cad94743,7A,CANDLELIGHT INN,2831 SW FAIRLAWN ROAD,TOPEKA,KS,66614,CoreFirst Bank & Trust,18100.0,NaN,3035 SW Topeka Blvd,TOPEKA,KS,66611,600000.0,510000.0,1990-10-01,1991,1990-10-31,OTH 7A,Guaranty,0.0,,180,,,None,,SHAWNEE,KS,KANSAS CITY DISTRICT OFFICE,01,PARTNERSHIP,,PIF,2005-07-31,NaT,0.0,0,0,
3,51def2b3-1e75-4992-9c84-819f858acaf8,7A,M.C. INTERNATIONAL INC,1100-B TUCKAHOE DRIVE,NASHVILLE,TN,37207,First Horizon Bank,4977.0,NaN,165 Madison Ave,MEMPHIS,TN,38103,96355.0,86720.0,1990-10-01,1991,1991-01-31,OTH 7A,Guaranty,0.0,,6,422330,"Women's, Children's, and Infants' Clothing and...",None,,DAVIDSON,TN,TENNESSEE DISTRICT OFFICE,05,CORPORATION,,PIF,2005-07-31,NaT,0.0,0,0,
4,6c6b7760-19c8-44b8-9501-7d8aeaab38cf,7A,Riv Inc.,36 CABOT ROAD,MERRIMACK,NH,03054,MISSINGMAINBANKID,NaN,NaN,,,,,120000.0,108000.0,1990-10-01,1991,None,CLP,Guaranty,0.0,,240,421830,Industrial Machinery and Equipment Wholesalers,None,,HILLSBOROUGH,NH,NEW HAMPSHIRE DISTRICT OFFICE,01,CORPORATION,,CANCLD,None,NaT,0.0,0,0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2790,f584032a-3e3d-45f3-aafe-6212cb375cf0,7A,INTERNATIONAL FILTER MANUFACTU,"2112 SCHRAM AVE., P.O. BOX 5",HILLSBORO,IL,62049,The Litchfield National Bank,3808.0,NaN,316 N State St,LITCHFIELD,IL,62056,76000.0,68400.0,1990-11-30,1991,None,OTH 7A,Guaranty,0.0,,84,,,None,,MONTGOMERY,IL,SPRINGFIELD BRANCH OFFICE,13,CORPORATION,,CANCLD,None,NaT,0.0,0,0,
2791,f5982a2b-4a0b-4229-8491-a6911202949b,7A,BELLINI JUNEVILE DESIGNER FURN,1126 E. PRADA COURT,SAN JOSE,CA,95120,Bank of Hope,26610.0,NaN,3200 Wilshire Blvd 1st Fl,LOS ANGELES,CA,90010,105000.0,94500.0,1990-11-30,1991,None,OTH 7A,Guaranty,0.0,,120,,,09700,BELLINI JUVENILE FURNITURE (7-,SANTA CLARA,CA,SAN FRANCISCO DISTRICT OFFICE,18,INDIVIDUAL,,CANCLD,None,NaT,0.0,0,0,
2792,faf75a94-161b-4570-8d3e-b3ca600735ed,7A,CUISINARY INC DBA ADRIANA'S BA,335 WEST 38 STREET,NEW YORK,NY,10018,Apple Bank for Savings,16068.0,NaN,1395 Northern Blvd,MANHASSET,NY,11030,100000.0,90000.0,1990-11-30,1991,1991-04-30,OTH 7A,Guaranty,0.0,,63,,,None,,NEW YORK,NY,NEW YORK DISTRICT OFFICE,10,CORPORATION,,CHGOFF,None,1999-01-27,119474.0,0,0,
2793,fb49bc34-188b-42b6-ab5a-92200c1ffb2e,7A,WINANS CONSTRUCTION INC.,"5515 DOYLE STREET, #9",EMERYVILLE,CA,94608,BMO Bank National Association,16571.0,NaN,320 S Canal St,Chicago,IL,60606,335600.0,268480.0,1990-11-30,1991,1991-01-31,PLP,Guaranty,0.0,,288,,,None,,ALAMEDA,CA,SAN FRANCISCO DISTRICT OFFICE,13,CORPORATION,,PIF,2005-07-31,NaT,0.0,0,0,Y


In [ ]:
# for quarter-based PD analysis
# Reset
origination_date = '1990-10-01'
origination_date_range = 3
MinGrossApproval = None
MaxGrossApproval = None
MinimumTermInMonths = None
NaicsCode = None
BorrState = None
BorrZip = None
BorrCity = None
BankName = None

all_parameter_data = []

def classify_quarter(origination_date):
    date = pd.to_datetime(origination_date)

    quarter = (date.month - 1) // 3 + 1

    return f"{date.year} Q{quarter}"

for quarter in range(1, 137):
    current_date = pd.to_datetime('1990-10-01') + DateOffset(months=3 * (quarter - 1))
    current_date_str = current_date.strftime('%Y-%m-%d')


    res_df = res_df_input(current_date_str,
                          origination_date_range,
                          engine,
                          MinGrossApproval=MinGrossApproval,
                          MaxGrossApproval=MaxGrossApproval,
                          MinimumTermInMonths=MinimumTermInMonths,
                          NaicsCode=NaicsCode,
                          BorrState=BorrState,
                          BorrZip=BorrZip,
                          BorrCity=BorrCity,
                          BankName=BankName)

    res_df['ChargeOffDate'] = pd.to_datetime(res_df['ChargeOffDate'])
    res_df['ApprovalDate'] = pd.to_datetime(res_df['ApprovalDate'])
    res_df = res_df.dropna(subset=['ApprovalDate'])

    results = []
    max_default_months = max_default_month(res_df)

    PD_month_array = PD_month(res_df, max_default_months + 1)

    for month in range(1, len(PD_month_array) + 1):
        month_data = PD_month_array[month - 1]
        results.append([
            int(month_data[0]),     # Month
            float(month_data[1]),   # Probability of Default
            int(month_data[2]),     # Defaulted Loans
            int(month_data[3]),     # Surviving Loans
            classify_quarter(current_date_str) # Quarter classification
        ])

    # Remove the last row if it contains zeros
    if len(results) > 0 and results[-1][2] == 0:
        results.pop()

    df_month = pd.DataFrame(results, columns=["Month", "Probability of Default", "Defaulted Loans", "Surviving Loans Until This Month", "Quarter"])
    all_parameter_data.append(df_month)

final_df = pd.concat(all_parameter_data, ignore_index=True)

# Save to CSV
print(final_df.to_csv('combined_PD_quarter.csv', index=False))

      Month  Probability of Default Defaulted Loans  \
0         1                 0.00000               0   
1         2                 0.00000               0   
2         3                 0.00000               0   
3         4                 0.00000               0   
4         5                 0.00000               0   
...     ...                     ...             ...   
24774     7                 0.00000               0   
24775     8                 0.00000               0   
24776     9                 0.00000               0   
24777    10                 0.00007               1   
24778    11                 0.00007               1   

      Surviving Loans Until This Month  Quarter  
0                                 4036  1990 Q4  
1                                 4036  1990 Q4  
2                                 4036  1990 Q4  
3                                 4036  1990 Q4  
4                                 4036  1990 Q4  
...                                ... 

/var/folders/7h/6t4d26817gvcypkm48pzm7p00000gn/T/ipykernel_56448/3138295488.py:72: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat(all_parameter_data, ignore_index=True)


In [ ]:
# for entire res_df

results = []
max_default_months = max_default_month(res_df)

PD_month_array = PD_month(res_df, max_default_months+1)

for month in range(1, len(PD_month_array)+1):
    month_data = PD_month_array[month-1]
    results.append([
    int(month_data[0]),     # Month
    float(month_data[1]),   # Probability of Default
    int(month_data[2]),     # Defaulted Loans
    int(month_data[3]),     # Surviving Loans
    ])

# solves problem of rounding
if len(results) > 0 and results[-1][2] == 0:
    results.pop()

df_month = pd.DataFrame(results, columns=["Month", "Probability of Default", "Defaulted Loans", "Surviving Loans Until This Month"])

print(df_month.to_csv('combined_PD.csv', index=False))

None


In [ ]:
def export_csv_based_on_parameter(res_df, parameter):
    unique_values = res_df[parameter].unique()
    all_parameter_data = []

    for value in unique_values:
        results = []
        filtered_res_df = res_df[res_df[parameter] == value]
        max_default_months = max_default_month(filtered_res_df)

        PD_month_array = PD_month(filtered_res_df, max_default_months+1)

        for month in range(1, len(PD_month_array)+1):
            month_data = PD_month_array[month-1]
            results.append([
            int(month_data[0]),     # Month
            float(month_data[1]),   # Probability of Default
            int(month_data[2]),     # Defaulted Loans
            int(month_data[3]),     # Surviving Loans
            value                   # Parameter Value
            ])

        # solves problem of rounding
        if len(results) > 0 and results[-1][2] == 0:
            results.pop()

        df_month = pd.DataFrame(results, columns=["Month", "Probability of Default", "Defaulted Loans", "Surviving Loans Until This Month", parameter])
        all_parameter_data.append(df_month)

    final_df = pd.concat(all_parameter_data, ignore_index=True)

    # return final_df
    name = 'combined_PD_' + parameter + '.csv'

    return final_df.to_csv(name, index=False)

In [ ]:
export_csv_based_on_parameter(res_df, 'BorrState')

/var/folders/7h/6t4d26817gvcypkm48pzm7p00000gn/T/ipykernel_31394/3149870288.py:29: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat(all_parameter_data, ignore_index=True)


In [ ]:
export_csv_based_on_parameter(res_df, 'BorrZip')

/var/folders/7h/6t4d26817gvcypkm48pzm7p00000gn/T/ipykernel_31394/3149870288.py:29: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat(all_parameter_data, ignore_index=True)


In [ ]:
export_csv_based_on_parameter(res_df, 'BorrCity')

/var/folders/7h/6t4d26817gvcypkm48pzm7p00000gn/T/ipykernel_31394/3149870288.py:29: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat(all_parameter_data, ignore_index=True)


In [ ]:
export_csv_based_on_parameter(res_df, 'BankName')

/var/folders/7h/6t4d26817gvcypkm48pzm7p00000gn/T/ipykernel_31394/3149870288.py:29: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat(all_parameter_data, ignore_index=True)


In [ ]:
def export_csv_based_on_parameter_naics(res_df, parameter):
    res_df = res_df.dropna(subset=[parameter])

    res_df['parameter_prefix'] = res_df[parameter].str[:2]
    unique_values = res_df['parameter_prefix'].unique()
    all_parameter_data = []

    for value in unique_values:
        results = []
        filtered_res_df = res_df[res_df['parameter_prefix'] == value]
        max_default_months = max_default_month(filtered_res_df)

        PD_month_array = PD_month(filtered_res_df, max_default_months+1)

        for month in range(1, len(PD_month_array)+1):
            month_data = PD_month_array[month-1]
            results.append([
            int(month_data[0]),     # Month
            float(month_data[1]),   # Probability of Default
            int(month_data[2]),     # Defaulted Loans
            int(month_data[3]),     # Surviving Loans
            value                   # Parameter Value
            ])

        # solves problem of rounding
        if len(results) > 0 and results[-1][2] == 0:
            results.pop()

        df_month = pd.DataFrame(results, columns=["Month", "Probability of Default", "Defaulted Loans", "Surviving Loans Until This Month", parameter])
        all_parameter_data.append(df_month)

    final_df = pd.concat(all_parameter_data, ignore_index=True)

    # return final_df
    name = 'combined_PD_' + parameter + '.csv'

    return final_df.to_csv(name, index=False)

In [ ]:
export_csv_based_on_parameter_naics(res_df, 'NaicsCode')

# Obtaining .csv's for pre-processed data (LGD)

In [ ]:
# reset
# change the parameters accordingly
origination_date = '1990-10-01'
origination_date_range = 408   # max num(months)
MinGrossApproval = None
MaxGrossApproval = None
MinimumTermInMonths = None
NaicsCode = None
BorrState = None
BorrZip = None
BorrCity = None
BankName = None

res_df = res_df_input(origination_date,
                      origination_date_range,
                      engine,
                      MinGrossApproval = MinGrossApproval,
                      MaxGrossApproval = MaxGrossApproval,
                      MinimumTermInMonths = MinimumTermInMonths,
                      NaicsCode = NaicsCode,
                      BorrState = BorrState,
                      BorrZip = BorrZip,
                      BorrCity = BorrCity,
                      BankName = BankName)
res_df

,LoanId,Program,BorrName,BorrStreet,BorrCity,BorrState,BorrZip,BankName,BankFDICNumber,BankNCUANumber,BankStreet,BankCity,BankState,BankZip,GrossApproval,SBAGuaranteedApproval,ApprovalDate,ApprovalFiscalYear,FirstDisbursementDate,DeliveryMethod,Subprogram,InitialInterestRate,FixedOrVariableInterestInd,TermInMonths,NaicsCode,NaicsDescription,FranchiseCode,FranchiseName,ProjectCounty,ProjectState,SBADistrictOffice,CongressionalDistrict,BusinessType,BusinessAge,LoanStatus,PaidInFullDate,ChargeOffDate,GrossChargeOffAmount,RevolverStatus,JobsSupported,SoldSecMrktInd
0,f64a607c-180e-4cf0-8a67-32a42059ea06,7A,TECHNIGRAPHICS,1880 PARKWAY BLVD.,WEST VALLEY,UT,84119,BMO Bank National Association,16571.0,NaN,320 S Canal St,Chicago,IL,60606,839200.0,671360.0,1990-10-01,1991,1990-10-31,PLP,Guaranty,0.00,,120,323121,Tradebinding and Related Work,None,,SALT LAKE,UT,UTAH DISTRICT OFFICE,02,PARTNERSHIP,,PIF,2005-07-31,None,0.0,0,0,Y
1,f5033e4e-35e8-49ec-8d07-03462cd60bd4,7A,REHABILITATION ADVISORS INC,445 WYMORE ROAD STE 220,WINTER PARK,FL,32789,"TD Bank, National Association",18409.0,NaN,2035 Limestone Rd,WILMINGTON,DE,19808,200000.0,170000.0,1990-10-01,1991,1991-01-31,OTH 7A,Guaranty,0.00,,60,624310,Vocational Rehabilitation Services,None,,ORANGE,FL,NORTH FLORIDA DISTRICT OFFICE,07,CORPORATION,,PIF,2005-07-31,None,0.0,0,0,
2,dbc26869-0981-49be-bb74-7021877ed905,7A,GREEN VALLEY GROCERY/CRAWFORD,3915 W. TWAIN,LAS VEGAS,NV,89103,"Wells Fargo Bank, National Association",3511.0,NaN,101 N Philips Ave,SIOUX FALLS,SD,57104,345000.0,258750.0,1990-10-01,1991,1990-10-31,PLP,Guaranty,0.00,,246,,,None,,CLARK,NV,NEVADA DISTRICT OFFICE,01,CORPORATION,,PIF,2005-07-31,None,0.0,0,0,N
3,dac354b6-721f-4f81-a84f-7463afee284f,7A,FOUR SEASONS PLANT CTR. INC.,222 S. POPE ST.,BRECKENRIDGE,MN,56520,BMO Bank National Association,16571.0,NaN,320 S Canal St,Chicago,IL,60606,98900.0,89010.0,1990-10-01,1991,1990-10-31,OTH 7A,Guaranty,0.00,,11,,,None,,WILKIN,MN,MINNESOTA DISTRICT OFFICE,07,CORPORATION,,PIF,2005-07-31,None,0.0,0,0,
4,c7b5804d-05a6-4d79-80bf-9e6f5bedab28,7A,Michael Davis,P O BOX 1457,FOND DU LAC,WI,54936,"U.S. Bank, National Association",6548.0,NaN,425 Walnut St,CINCINNATI,OH,45202,250000.0,200000.0,1990-10-01,1991,1990-10-31,PLP,Guaranty,0.00,,48,,,None,,FOND DU LAC,WI,MILWAUKEE,06,CORPORATION,,PIF,2005-07-31,None,0.0,0,0,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1803628,a6f09f19-2311-4cd1-adb9-3d7d2e49d4af,7A,Saunders Construction LLC,18 UNION ST,READING,MA,01867,Rockland Trust Company,9712.0,NaN,288 Union St,ROCKLAND,MA,02370,75000.0,37500.0,2024-03-30,2024,None,SBA EXPRES,FA$TRK (Small Loan Express),9.50,V,120,238190,"Other Foundation, Structure, and Building Exte...",None,,MIDDLESEX,MA,MASSACHUSETTS DISTRICT OFFICE,06,CORPORATION,Existing or more than 2 years old,COMMIT,None,None,0.0,1,5,
1803629,a5e265c8-5d70-4588-ae25-1d061f156786,7A,OPEN RETAIL INC.,108 N ST ANDREWS PL,LOS ANGELES,CA,90004,BayFirst National Bank,34997.0,NaN,700 Central Avenue,Saint Petersburg,FL,33701,150000.0,127500.0,2024-03-30,2024,None,PLP,Guaranty,13.25,V,120,513210,Cable Networks,None,,LOS ANGELES,CA,LOS ANGELES DISTRICT OFFICE,28,CORPORATION,Existing or more than 2 years old,COMMIT,None,None,0.0,0,2,
1803630,b3c0fbe7-7d71-4b19-ae49-c2d83d4dee97,7A,MY DRYWALL LLC,9191 STATE ROUTE 305,GARRETTSVILLE,OH,44231,The Huntington National Bank,6560.0,NaN,17 S High St.,COLUMBUS,OH,43215,150000.0,75000.0,2024-03-30,2024,None,SBA EXPRES,FA$TRK (Small Loan Express),10.50,V,120,238310,Drywall and Insulation Contractors,None,,PORTAGE,OH,CLEVELAND DISTRICT OFFICE,13,CORPORATION,Existing or more than 2 years old,COMMIT,None,None,0.0,1,0,
1803631,b4640bb3-13c4-4f70-971a-195b2368dd4d,7A,Green Turf Owners Landscaping,418 STAHL AVE,NEW CASTLE,DE,19720,"TD Bank, National Association",18409.0,NaN,2035 Limestone Rd,WILMINGTON,DE,19808,25000.0,12500.0,2024-03-30,2024,None,SBA 

In [ ]:
from google.colab import files

def export_LGD(res_df, parameter):
    chgoff_df = res_df[res_df['LoanStatus'] == 'CHGOFF']

    grouped = chgoff_df.groupby(parameter)['GrossChargeOffAmount'].agg(['mean', 'std', 'min', 'max', 'count']).reset_index()


    grouped['std'] = grouped['std'].fillna('N/A')

    grouped['mean'] = grouped['mean'].round(2)
    grouped['std'] = grouped['std'].apply(lambda x: round(x, 2) if x != 'N/A' else x)

    grouped.columns = [parameter, 'Mean', 'StdDev', 'Min', 'Max', 'Total Number']

    name = f'combined_LGD_{parameter}.csv'
    return grouped.to_csv(name, index=False)

# res_df['parameter_prefix'] = res_df[parameter].str[:2]
#     unique_values = res_df['parameter_prefix'].unique()
#     all_parameter_data = []

#     for value in unique_values:
#         filtered_res_df = res_df[res_df['parameter_prefix'] == value]

In [ ]:
def export_LGD_quarter(res_df):
    # Filter the DataFrame once for 'CHGOFF' LoanStatus
    chgoff_df = res_df[res_df['LoanStatus'] == 'CHGOFF']

    if chgoff_df.empty:
        mean = 'N/A'
        std_dev = 'N/A'
        min_value = 'N/A'
        max_value = 'N/A'
        length = 0
    else:
        gross_CHGOFF = chgoff_df['GrossChargeOffAmount']
        mean = round(gross_CHGOFF.mean(), 2)
        std_dev = round(gross_CHGOFF.std(), 2) if len(gross_CHGOFF) > 1 else 'N/A'
        min_value = gross_CHGOFF.min()
        max_value = gross_CHGOFF.max()
        length = gross_CHGOFF.size

    return [mean, std_dev, min_value, max_value, length]

In [ ]:
def export_LGD_quarter_total(res_df):
    chgoff_df = res_df[res_df['LoanStatus'] == 'CHGOFF']

    gross_CHGOFF = chgoff_df['GrossChargeOffAmount']
    mean = round(gross_CHGOFF.mean(), 2)
    std_dev = round(gross_CHGOFF.std(), 2) if len(gross_CHGOFF) > 1 else 'N/A'
    min_value = gross_CHGOFF.min()
    max_value = gross_CHGOFF.max()
    length = gross_CHGOFF.size

    data = {
        'Mean': [mean],
        'StdDev': [std_dev],
        'Min': [min_value],
        'Max': [max_value],
        'Total Number': [length]
    }
    result_df = pd.DataFrame(data)

    # Save to CSV
    return(result_df.to_csv('combined_LGD.csv', index=False))

In [ ]:
print(export_LGD_quarter_total(res_df))
files.download('combined_LGD.csv')

None


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
print(export_LGD(res_df, 'BorrCity'))
files.download('combined_LGD_BorrCity.csv')

None


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# for quarter-based PD analysis
# Reset
origination_date = '1990-10-01'
origination_date_range = 3
MinGrossApproval = None
MaxGrossApproval = None
MinimumTermInMonths = None
NaicsCode = None
BorrState = None
BorrZip = None
BorrCity = None
BankName = None

all_parameter_data = []

def classify_quarter(origination_date):

    date = pd.to_datetime(origination_date)

    quarter = (date.month - 1) // 3 + 1

    return f"{date.year} Q{quarter}"

for quarter in range(1, 137):
    current_date = pd.to_datetime(origination_date) + DateOffset(months=3 * (quarter - 1))
    current_date_str = current_date.strftime('%Y-%m-%d')

    res_df = res_df_input(current_date_str,
                          origination_date_range,
                          engine,
                          MinGrossApproval=MinGrossApproval,
                          MaxGrossApproval=MaxGrossApproval,
                          MinimumTermInMonths=MinimumTermInMonths,
                          NaicsCode=NaicsCode,
                          BorrState=BorrState,
                          BorrZip=BorrZip,
                          BorrCity=BorrCity,
                          BankName=BankName)

    # Process the DataFrame
    res_df['ChargeOffDate'] = pd.to_datetime(res_df['ChargeOffDate'])
    res_df['ApprovalDate'] = pd.to_datetime(res_df['ApprovalDate'])
    res_df = res_df.dropna(subset=['ApprovalDate'])

    # Perform LGD calculation
    lgd_results = export_LGD_quarter(res_df)
    lgd_results.append(classify_quarter(current_date_str))

    all_parameter_data.append(lgd_results)

final_df = pd.DataFrame(all_parameter_data, columns=['Mean', 'StdDev', 'Min', 'Max', 'Total Number', 'Quarter'])

# Save to CSV
final_df.to_csv('combined_LGD_quarter.csv', index=False)
print(final_df)
files.download('combined_LGD_quarter.csv')

In [ ]:
caption = ''

variables = [
    ("Origination Date", origination_date),
    ("Origination Date Range", origination_date_range),
    ("Min Gross Approval", MinGrossApproval),
    ("Max Gross Approval", MaxGrossApproval),
    ("Min Term (Months)", MinimumTermInMonths),
    ("NAICS Code", NaicsCode),
    ("Borrower State", BorrState),
    ("Borrower Zip", BorrZip),
    ("Borrower City", BorrCity),
    ("Bank Name", BankName)
]

for desc, var in variables:
    if var is not None:
        caption += f'{desc}: {var} | '

bin_size = 20000
num_bins = int((max(gross_CHGOFF) - min(gross_CHGOFF)) // bin_size) + 1
if num_bins < 10:
    num_bins = 10
if num_bins > 75:
    num_bins = 75

plt.hist(gross_CHGOFF, bins=num_bins, edgecolor='black')

caption = caption.rstrip(' | ')
wrapped_caption = textwrap.fill(caption, width=100)

fixed_width = 80
fixed_height = 4
wrapped_caption = textwrap.fill(caption, width=fixed_width)

lines_in_caption = wrapped_caption.count('\n') + 1
font_size = 8 #inital font size

while lines_in_caption > fixed_height and font_size > 6:
    font_size -= 0.5
    wrapped_caption = textwrap.fill(caption, width=fixed_width)
    lines_in_caption = wrapped_caption.count('\n') + 1

plt.title('Defaulted Loan Size')
plt.xlabel('Loan Size')
plt.ylabel('Frequency')

plt.gca().yaxis.set_major_locator(MaxNLocator(integer=True))

plt.subplots_adjust(bottom=0.18)

plt.figtext(0.515, -0.03, wrapped_caption, ha="center", fontsize=font_size, bbox={"facecolor": "lightgrey", "alpha": 0.5, "pad": 5})

plt.show()

# 5y Cumulative Default for Hotels (NaicsCode=721110)

In [ ]:
# reset
# change the parameters accordingly
origination_date = '1990-10-01'
origination_date_range = 408   # max num(months)
MinGrossApproval = None
MaxGrossApproval = None
MinimumTermInMonths = None
NaicsCode = '72'
BorrState = None
BorrZip = None
BorrCity = None
BankName = None

res_df = res_df_input(origination_date,
                      origination_date_range,
                      engine,
                      MinGrossApproval = MinGrossApproval,
                      MaxGrossApproval = MaxGrossApproval,
                      MinimumTermInMonths = MinimumTermInMonths,
                      NaicsCode = NaicsCode,
                      BorrState = BorrState,
                      BorrZip = BorrZip,
                      BorrCity = BorrCity,
                      BankName = BankName)
res_df

,LoanId,Program,BorrName,BorrStreet,BorrCity,BorrState,BorrZip,BankName,BankFDICNumber,BankNCUANumber,BankStreet,BankCity,BankState,BankZip,GrossApproval,SBAGuaranteedApproval,ApprovalDate,ApprovalFiscalYear,FirstDisbursementDate,DeliveryMethod,Subprogram,InitialInterestRate,FixedOrVariableInterestInd,TermInMonths,NaicsCode,NaicsDescription,FranchiseCode,FranchiseName,ProjectCounty,ProjectState,SBADistrictOffice,CongressionalDistrict,BusinessType,BusinessAge,LoanStatus,PaidInFullDate,ChargeOffDate,GrossChargeOffAmount,RevolverStatus,JobsSupported,SoldSecMrktInd
0,520514df-7abd-43de-a269-3947d3836ce2,7A,FRED'S FRIENDS OF EDWARDSVILLE,"157 CENTER, OUTLOT #2",EDWARDSVILLE,IL,62025,"U.S. Bank, National Association",6548.0,NaN,425 Walnut St,CINCINNATI,OH,45202,460000.0,368000.0,1990-10-03,1991,1991-04-30,PLP,Guaranty,0.00,,239,722410,Drinking Places (Alcoholic Beverages),None,,MADISON,IL,SPRINGFIELD BRANCH OFFICE,12,CORPORATION,,PIF,2011-09-30,None,0.0,0,0,Y
1,29e2e39e-97f2-4ba1-97a3-a01057184a08,7A,KING CREEK RESORT INC.,"RT. 4, BOX 87",BENTON,KY,42025,"Bank of America, National Association",3510.0,NaN,"100 N Tryon St, Ste 170",CHARLOTTE,NC,28202,220000.0,179520.0,1990-10-16,1991,1991-01-31,CLP,Guaranty,0.00,,144,721214,Recreational and Vacation Camps (except Campgr...,None,,MARSHALL,KY,TENNESSEE DISTRICT OFFICE,01,CORPORATION,,PIF,2005-07-31,None,0.0,0,0,Y
2,97238700-6d03-417a-a2b0-160aa4449a55,7A,CLUB WEST COAST/MAGNUSON,2028 HANCOCK STREET,SAN DIEGO,CA,92110,"U.S. Bank, National Association",6548.0,NaN,425 Walnut St,CINCINNATI,OH,45202,868500.0,694800.0,1990-10-25,1991,1991-01-31,PLP,Guaranty,0.00,,244,722410,Drinking Places (Alcoholic Beverages),None,,SAN DIEGO,CA,SAN DIEGO DISTRICT OFFICE,52,INDIVIDUAL,,CHGOFF,None,1997-07-21,288236.0,0,0,Y
3,e14c987e-9a31-4cec-b0d3-966f4c3582f9,7A,STINGRAYS BAR & GRILL,AV 134 HWY 51 NORTH,WOODRUFF,WI,54568,BMO Bank National Association,16571.0,NaN,320 S Canal St,Chicago,IL,60606,6000.0,5400.0,1990-10-25,1991,1991-07-31,CLP,Guaranty,0.00,,120,722410,Drinking Places (Alcoholic Beverages),None,,ONEIDA,WI,WISCONSIN DISTRICT OFFICE,07,CORPORATION,,PIF,2005-07-31,None,0.0,0,0,
4,618adaf0-3b2a-4ad3-a746-d82e4d184fb4,7A,STINGRAYS BAR & GRILL,AV 134 HWY 51 NORTH,WOODRUFF,WI,54568,BMO Bank National Association,16571.0,NaN,320 S Canal St,Chicago,IL,60606,120250.0,108225.0,1990-10-25,1991,1991-07-31,CLP,Guaranty,0.00,,240,722410,Drinking Places (Alcoholic Beverages),None,,ONEIDA,WI,WISCONSIN DISTRICT OFFICE,07,CORPORATION,,PIF,2005-07-31,None,0.0,0,0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187784,69ad3ee1-5d29-4111-9fcb-db056e2a2b64,7A,YPP HOLDINGS LLC,3630 PEACHTREE PKWY STE 703,SUWANEE,GA,30024,The Huntington National Bank,6560.0,NaN,17 S High St.,COLUMBUS,OH,43215,30000.0,15000.0,2024-03-30,2024,None,SBA EXPRES,FA$TRK (Small Loan Express),11.00,V,120,722513,Limited-Service Restaurants,S2089,Clean Eatz,FORSYTH,GA,GEORGIA DISTRICT OFFICE,06,CORPORATION,"Startup, Loan Funds will Open Business",COMMIT,None,None,0.0,1,24,
187785,7ce381bd-3e56-4db3-b97b-229e22e193e9,7A,Tri City Hotels LLC,6604 LONDONDERRY WAY,UNION CITY,GA,30291,Commonwealth Business Bank,57873.0,NaN,"3435 Wilshire Blvd, Ste 700",LOS ANGELES,CA,90010,550000.0,412500.0,2024-03-30,2024,None,PLP,Guaranty,9.50,V,300,721110,Hotels (except Casino Hotels) and Motels,S0982,La Quinta Inn,FULTON,GA,GEORGIA DISTRICT OFFICE,13,CORPORATION,Existing or more than 2 years old,COMMIT,None,None,0.0,0,10,
187786,0fb73368-1d0d-4a7a-a990-60831b407173,7A,3 LNL Inc.,360 S Heritage Way Suite 400,ANDOVER,KS,67002,Equity Bank,25858.0,NaN,345 N Andover Rd,ANDOVER,KS,67002,337500.0,253125.0,2024-03-30,2024,None,PLP,Guaranty,10.50,V,126,722511,Full-Service Restaurants,None,,BUTLER,KS,WICHITA DISTRICT OFFICE,04,CORPORATION,"Startup, Loan Funds will Open Business",COMMIT,None,None,0.0,0,25,
187787,045b6f72-176b-4633-ae4d-f9308eed2c0e,7A,Santar

In [ ]:
from google.colab import files

all_parameter_data = []

res_df['ChargeOffDate'] = pd.to_datetime(res_df['ChargeOffDate'])
res_df['ApprovalDate'] = pd.to_datetime(res_df['ApprovalDate'])

def classify_quarter(origination_date):
    date = pd.to_datetime(origination_date)
    quarter = (date.month - 1) // 3 + 1
    return f"{date.year} Q{quarter}"

for quarter in range(1, 138):

    current_date = pd.to_datetime('1990-10-01') + DateOffset(months=3 * (quarter - 1))
    current_date_str = current_date.strftime('%Y-%m-%d')

    res_df = res_df_input(
        current_date_str,
        3,
        engine,
        MinGrossApproval=MinGrossApproval,
        MaxGrossApproval=MaxGrossApproval,
        MinimumTermInMonths=MinimumTermInMonths,
        NaicsCode=NaicsCode,
        BorrState=BorrState,
        BorrZip=BorrZip,
        BorrCity=BorrCity,
        BankName=BankName
    )

    res_df_hotel = res_df[res_df['NaicsCode'] == '721110']

    res_df_hotel = res_df_hotel.dropna(subset=['ApprovalDate'])

    res_df_hotel['ChargeOffDate'] = pd.to_datetime(res_df_hotel['ChargeOffDate'])
    res_df_hotel['ApprovalDate'] = pd.to_datetime(res_df_hotel['ApprovalDate'])

    five_year_cutoff = current_date + DateOffset(years=5)


    res_df_hotel_default = res_df_hotel[
        res_df_hotel['ChargeOffDate'].notnull() &
        (res_df_hotel['ApprovalDate'] <= five_year_cutoff)
    ]

    if len(res_df_hotel) > 0:
        default_probability = len(res_df_hotel_default) / len(res_df_hotel)
    else:
        default_probability = 0

    results = [
        default_probability,                   # Probability of Default within 5 Years
        len(res_df_hotel_default),             # Defaulted Loans within 5 Years
        len(res_df_hotel),                     # Total Loans
        classify_quarter(current_date_str)     # Quarter classification
    ]

    df_month = pd.DataFrame([results], columns=["Probability of Default within 5 Years", "Defaulted Loans within 5 Years", "Total Loans", "Quarter"])
    all_parameter_data.append(df_month)

final_df = pd.concat(all_parameter_data, ignore_index=True)

final_df.to_csv('721110_5y_cumulative_default.csv', index=False)
print(final_df)
files.download('721110_5y_cumulative_default.csv')

     Probability of Default within 5 Years  Defaulted Loans within 5 Years  \
0                                      0.0                               0   
1                                      0.0                               0   
2                                      0.0                               0   
3                                      0.0                               0   
4                                      0.0                               0   
..                                     ...                             ...   
132                                    0.0                               0   
133                                    0.0                               0   
134                                    0.0                               0   
135                                    0.0                               0   
136                                    0.0                               0   

     Total Loans  Quarter  
0              0  1990 Q4  
1      

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>